<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=ad9689c20f6376a53a4e420567760597ee0d2f1f90150d44c7ca22386470b43b
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0


    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-10 14:44:02
-------------------
qualified stocks: 87
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.05 L
Current:  1.43 C
-------------------
Today PnL: 35.99 K (0.25%)
Current PnL: -21.40 L (-14.02%)
CY Booked + Current PnL: -8.75 L (-5.73%)
-------------------
Total profit:  2.27 L
Total loss:  -23.67 L
-------------------
Total Booked + Current PnL: 18.23 L (14.51%)
Total Booked PnL: 39.63 L (31.55%)
Curr Year Booked PnL: 12.65 L (8.84%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 88.38 L (61.73%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
58,RELIANCE,1533.00,-6.91,64.0,X-LC,6.74,232690.0,22324.0,6469.0,0.92,10.61,2.78,13.68,37.0,3.45,1.64,28.89,XY25,NTT,REFINERIES
79,UNIONBANK,163.00,-2.33,69.0,M-LC,11.56,175826.0,34986.0,9512.0,0.87,24.84,5.41,31.59,66.0,3.68,1.24,57.91,XY24,NTT,BANKS
78,TTKPRESTIG,770.00,101.07,51.0,M-SC,2.01,86670.0,-14107.0,14197.0,-1.01,-14.00,16.38,0.09,245.0,-0.99,0.61,13.86,OX40N,NTT,DURABLES
33,ICICIGI,2252.93,-12.68,60.0,X-MC,6.28,150213.0,14080.0,16508.0,0.48,10.34,10.99,22.47,91.0,0.85,1.06,24.66,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,-10.25,46.0,X-MC,3.04,88575.0,-2103.0,17006.0,2.57,-2.32,19.20,16.44,101.0,-0.12,0.62,17.39,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIGOPNTS,1408.0,122.66,74.0,M-SC,5.70,145936.0,-28623.0,28662.0,17.21,-16.40,19.64,0.02,221.0,-1.00,1.03,26.94,OX40N,NTT,PAINTS
19,COFFEEDAY,80.0,-47.69,44.0,L-SC,27.38,77856.0,-35693.0,75746.0,4.00,-31.43,97.29,35.27,268.0,-0.47,0.55,90.55,XR,NTT,HOTELS
61,SAIL,228.0,58.78,71.0,M-MC,10.05,250034.0,25072.0,142569.0,2.98,11.14,57.02,74.53,192.0,0.18,1.76,46.99,XY24,BTT,STEEL
39,INFY,2275.0,-17.09,61.0,X-LC,4.86,323611.0,10723.0,160964.0,2.88,3.43,49.74,54.87,3.0,0.07,2.28,11.99,X40,BTT,IT
2,ABBOTINDIA,35195.0,-10.25,46.0,X-MC,3.04,88575.0,-2103.0,17006.0,2.57,-2.32,19.20,16.44,101.0,-0.12,0.62,17.39,X40,ATH,PHARMA


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,240.00,49.12,35.0,H-SC,14.41,136629.0,-42442.0,108169.0,-6.51,-23.70,79.17,36.71,135.0,-0.39,0.96,35.56,XR,NTT,BANKS
65,SHALBY,327.00,1053.11,35.0,M-SC,17.11,156722.0,-24421.0,69882.0,-3.97,-13.48,44.59,25.10,235.0,-0.35,1.10,25.60,XY24,NTT,HEALTHCARE
85,WHIRLPOOL,2270.00,-42.31,46.0,M-SC,7.03,119600.0,4802.0,89246.0,-2.99,4.18,74.62,81.92,223.0,0.05,0.84,41.59,XR,NTT,DURABLES
46,KPIGREEN,731.05,3.26,45.0,H-SC,9.44,123215.0,-2082.0,61004.0,-2.83,-1.66,49.51,47.03,141.0,-0.03,0.87,54.28,MH,ATH,POWER
55,RAJOOENG,143.10,-43.93,41.0,H-SC,16.69,85040.0,-17460.0,58057.0,-2.31,-17.03,68.27,39.61,114.0,-0.30,0.60,3.86,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.07,51.0,M-SC,2.01,86670.0,-14107.0,14197.0,-1.01,-14.00,16.38,0.09,245.0,-0.99,0.61,13.86,OX40N,NTT,DURABLES
37,INDIGOPNTS,1408.0,122.66,74.0,M-SC,5.70,145936.0,-28623.0,28662.0,17.21,-16.40,19.64,0.02,221.0,-1.00,1.03,26.94,OX40N,NTT,PAINTS
45,KANSAINER,340.0,-67.79,46.0,H-SC,2.36,222165.0,-47502.0,83845.0,0.22,-17.62,37.74,13.47,138.0,-0.57,1.56,12.96,XY24,NTT,PAINTS
67,SIS,528.0,2026.42,48.0,H-SC,2.33,85808.0,-25224.0,48833.0,1.17,-22.72,56.91,21.26,156.0,-0.52,0.60,15.93,OX40N,NTT,MISC
17,CERA,9475.0,-25.54,38.0,H-SC,1.87,135930.0,-39973.0,81993.0,-1.45,-22.72,60.32,23.89,149.0,-0.49,0.96,17.39,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,UNIONBANK,163.0,-2.33,69.0,M-LC,11.56,175826.0,34986.0,9512.0,0.87,24.84,5.41,31.59,66.0,3.68,1.24,57.91,XY24,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,58.78,71.0,M-MC,10.05,250034.0,25072.0,142569.0,2.98,11.14,57.02,74.53,192.0,0.18,1.76,46.99,XY24,BTT,STEEL
31,HINDZINC,730.22,29.16,54.0,M-LC,9.41,210975.0,5899.0,106669.0,2.21,2.88,50.56,54.89,52.0,0.06,1.48,27.67,X5K,ATH,METALS
79,UNIONBANK,163.00,-2.33,69.0,M-LC,11.56,175826.0,34986.0,9512.0,0.87,24.84,5.41,31.59,66.0,3.68,1.24,57.91,XY24,NTT,BANKS
85,WHIRLPOOL,2270.00,-42.31,46.0,M-SC,7.03,119600.0,4802.0,89246.0,-2.99,4.18,74.62,81.92,223.0,0.05,0.84,41.59,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.52,52.0,H-SC,5.25,129744.0,6408.0,125216.0,0.02,5.20,96.51,106.72,119.0,0.05,0.91,29.79,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,29.16,54.0,M-LC,9.41,210975.0,5899.0,106669.0,2.21,2.88,50.56,54.89,52.0,0.06,1.48,27.67,X5K,ATH,METALS
85,WHIRLPOOL,2270.00,-42.31,46.0,M-SC,7.03,119600.0,4802.0,89246.0,-2.99,4.18,74.62,81.92,223.0,0.05,0.84,41.59,XR,NTT,DURABLES
36,INDIAMART,4810.62,-53.52,52.0,H-SC,5.25,129744.0,6408.0,125216.0,0.02,5.20,96.51,106.72,119.0,0.05,0.91,29.79,AR,ATH,MISC
86,WIPRO,420.00,-15.15,46.0,M-LC,5.59,149060.0,-1885.0,111333.0,1.66,-1.25,74.69,72.51,53.0,-0.02,1.05,4.70,XR,NTT,IT
46,KPIGREEN,731.05,3.26,45.0,H-SC,9.44,123215.0,-2082.0,61004.0,-2.83,-1.66,49.51,47.03,141.0,-0.03,0.87,54.28,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TATAMOTORS,1065.00,-53.17,22.0,X-LC,34.23,147269.0,-129254.0,239327.0,-0.53,-46.74,162.51,39.81,54.0,-0.54,1.04,3.80,XY24,NTT,AUTO
44,JSWINFRA,342.00,-24.98,25.0,X-MC,4.42,188359.0,-11988.0,42494.0,-0.41,-5.98,22.56,15.23,178.0,-0.28,1.33,24.34,X40N,NTT,REALTY
23,DMART,5391.45,-21.15,26.0,X-LC,7.16,201005.0,-9402.0,68563.0,0.23,-4.47,34.11,28.12,38.0,-0.14,1.41,18.10,X40N,ATH,FMCG
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.15,87150.0,-41520.0,86819.0,-1.12,-32.27,99.62,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
30,HINDUNILVR,2922.00,-14.27,31.0,X-LC,6.22,240810.0,-16253.0,51389.0,-0.24,-6.32,21.34,13.67,24.0,-0.32,1.69,13.68,XY25,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,PGHH,17907.65,-31.34,37.0,X-MC,0.28,198750.0,-2070.0,69861.0,0.58,-1.03,35.15,33.76,80.0,-0.03,1.40,4.11,X40,ATH,FMCG
41,ITC,452.00,-38.77,42.0,X-LC,1.08,197580.0,-2558.0,23453.0,-0.85,-1.28,11.87,10.44,4.0,-0.11,1.39,4.21,X40,NTT,FMCG
66,SIEMENS,4671.50,-4.54,43.0,H-LC,1.54,153550.0,-32545.0,80030.0,1.24,-17.49,52.12,25.51,15.0,-0.41,1.08,13.15,AR,ATH,ELECTRICAL
28,HAVELLS,2069.16,-8.02,40.0,X-MC,3.27,306726.0,-21265.0,127813.0,0.84,-6.48,41.67,32.48,92.0,-0.17,2.16,4.20,X40,ATH,ELECTRICAL
80,UNITDSPR,1644.00,-7.52,58.0,X-LC,3.81,242262.0,8118.0,40506.0,-1.44,3.47,16.72,20.77,86.0,0.20,1.70,10.63,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.15,87150.0,-41520.0,86819.0,-1.12,-32.27,99.62,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
53,QUESS,986.00,-50.41,35.0,X-SC,37.34,49163.0,-15843.0,166771.0,-0.91,-24.37,339.22,232.18,198.0,-0.09,0.35,0.62,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.00,38.0,X-MC,7.53,80280.0,-3230.0,23257.0,1.01,-3.87,28.97,23.98,82.0,-0.14,0.56,1.73,X40,ATH,APPARELS
20,COLPAL,3726.84,-6.96,41.0,X-MC,7.18,217400.0,-45965.0,155289.0,0.31,-17.45,71.43,41.51,84.0,-0.30,1.53,2.02,XY25,ATH,FMCG
1,ABB,7934.00,-42.59,33.0,H-LC,4.95,239808.0,-21811.0,141031.0,-0.41,-8.34,58.81,45.57,7.0,-0.15,1.69,2.04,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-50.41,35.0,X-SC,37.34,49163.0,-15843.0,166771.0,-0.91,-24.37,339.22,232.18,198.0,-0.09,0.35,0.62,XY24,NTT,MISC
51,PAGEIND,51769.93,-29.00,38.0,X-MC,7.53,80280.0,-3230.0,23257.0,1.01,-3.87,28.97,23.98,82.0,-0.14,0.56,1.73,X40,ATH,APPARELS
57,RELAXO,1176.00,-41.04,39.0,X-SC,4.22,80342.0,-64818.0,140743.0,-0.95,-44.65,175.18,52.31,136.0,-0.46,0.57,7.54,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.15,87150.0,-41520.0,86819.0,-1.12,-32.27,99.62,35.20,219.0,-0.48,0.61,0.03,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.25,46.0,X-MC,3.04,88575.0,-2103.0,17006.0,2.57,-2.32,19.20,16.44,101.0,-0.12,0.62,17.39,X40,ATH,PHARMA


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.69,52.0,X-LC,9.55,284782.0,-61174.0,127867.0,1.26,-17.68,44.90,19.28,1.0,-0.48,2.00,5.28,X40,ATH,IT
39,INFY,2275.00,-17.09,61.0,X-LC,4.86,323611.0,10723.0,160964.0,2.88,3.43,49.74,54.87,3.0,0.07,2.28,11.99,X40,BTT,IT
41,ITC,452.00,-38.77,42.0,X-LC,1.08,197580.0,-2558.0,23453.0,-0.85,-1.28,11.87,10.44,4.0,-0.11,1.39,4.21,X40,NTT,FMCG
83,VBL,671.64,-17.33,50.0,X-LC,5.65,297468.0,-18374.0,131035.0,-0.84,-5.82,44.05,35.67,5.0,-0.14,2.09,7.23,X40N,ATH,FMCG
1,ABB,7934.00,-42.59,33.0,H-LC,4.95,239808.0,-21811.0,141031.0,-0.41,-8.34,58.81,45.57,7.0,-0.15,1.69,2.04,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,44.0,L-SC,27.38,77856.0,-35693.0,75746.0,4.00,-31.43,97.29,35.27,268.0,-0.47,0.55,90.55,XR,NTT,HOTELS
7,ASIANTILES,137.00,7361.11,63.0,L-SC,12.98,81868.0,-11942.0,88565.0,0.12,-12.73,108.18,81.67,269.0,-0.13,0.58,58.54,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.15,49.0,H-SC,7.97,91875.0,-6105.0,27710.0,-0.36,-6.23,30.16,22.05,152.0,-0.22,0.65,33.55,XR,ATH,FINANCE
46,KPIGREEN,731.05,3.26,45.0,H-SC,9.44,123215.0,-2082.0,61004.0,-2.83,-1.66,49.51,47.03,141.0,-0.03,0.87,54.28,MH,ATH,POWER
37,INDIGOPNTS,1408.00,122.66,74.0,M-SC,5.70,145936.0,-28623.0,28662.0,17.21,-16.40,19.64,0.02,221.0,-1.00,1.03,26.94,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VAIBHAVGBL,521.00,70.80,61.0,H-SC,4.81,148135.0,-34640.0,145706.0,-1.20,-18.95,98.36,60.77,125.0,-0.24,1.04,33.41,XR,NTT,JEWELLERY
5,ANGELONE,3033.00,21.43,67.0,X-SC,6.44,215078.0,24072.0,33638.0,0.25,12.60,15.64,30.21,157.0,0.72,1.51,36.12,X40N,NTT,FINANCE
6,ASIANPAINT,3465.66,-7.57,74.0,X-LC,5.61,233262.0,-18506.0,71728.0,1.41,-7.35,30.75,21.13,27.0,-0.26,1.64,25.47,X40,ATH,PAINTS
68,SONACOMS,806.63,-31.14,69.0,M-SC,6.26,85785.0,-15374.0,55374.0,-0.18,-15.20,64.55,39.54,202.0,-0.28,0.60,20.97,AR,ATH,AUTO
7,ASIANTILES,137.00,7361.11,63.0,L-SC,12.98,81868.0,-11942.0,88565.0,0.12,-12.73,108.18,81.67,269.0,-0.13,0.58,58.54,XR,NTT,CERAMICS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.80
1,25,44.54
2,50,75.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.65
LC    33.22
MC    23.12
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.98
X40      16.40
X40N     12.40
XY25     11.91
XR        9.75
AR        7.77
OX40N     6.61
X200      1.79
X5K       1.48
SR        1.03
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.01
X-LC    22.50
X-MC    18.72
M-SC    12.60
X-SC     5.58
M-LC     5.17
H-LC     4.56
H-MC     2.32
M-MC     1.76
L-SC     1.46
L-LC     0.99
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.18,-4.84,38.73
IT,12.57,-17.71,80.71
FINANCE,12.14,-9.54,59.20
MISC,7.76,-17.83,77.27
PAINTS,5.76,-12.81,29.63
ELECTRICAL,5.69,-11.80,51.44
BANKS,3.96,-20.03,89.86
INSURANCE,3.92,0.74,36.27
AUTO,2.71,-49.38,111.73


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3314870.0,22
XR,1271416.0,13
AR,1182934.0,9
X40,824355.0,12
X40N,705253.0,9
OX40N,545820.0,9
XY25,480299.0,8
SR,275093.0,2
X5K,106669.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3417092.0,24
M-SC,1255948.0,15
X-MC,1213935.0,14
X-LC,1079456.0,13
X-SC,556141.0,6
H-LC,290923.0,3
M-LC,273269.0,4
H-MC,258393.0,2
L-SC,254595.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1188022.0      6
           AR         838228.0      5
           XR         800095.0      7
M-SC       XY24       737192.0      6
X-MC       XY24       575013.0      4
X-LC       X40        443762.0      5
           XY24       312053.0      2
X-SC       X40N       302551.0      4
X-MC       X40        293774.0      6
H-SC       SR         275093.0      2
           OX40N      254650.0      3
X-LC       X40N       240104.0      3
H-LC       AR         221061.0      2
H-MC       XY24       183738.0      1
X-MC       XY25       182550.0      2
X-SC       XY24       166771.0      1
L-SC       XR         164311.0      2
X-MC       X40N       162598.0      2
M-MC       XY24       142569.0      1
M-SC       XR         138718.0      2
           XY25       130162.0      1
           OX40N      126231.0      4
           AR         123645.0      2
M-LC       XR         111333.0      1
           X5K        106669.0      1
L-SC       OX40N       90284.0      1
X-SC       X40         86819.0      1
X-LC       XY25        83537.0      3
H-MC       OX40N       74655.0      1
H-LC       X200        69862.0      1
H-SC       MH          61004.0      1
L-MC       XR          56959.0      1
M-LC       XY25        45755.0      1
L-LC       XY25        38295.0      1
M-LC       XY24         9512.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
